In [2]:
import pandas as pd
from pathlib import Path

# Files you already shared
UHC_CSV = Path("GA_UHC.csv")
GA_WC_FLAT = Path("ga_wc_flat.csv")



In [3]:
# NEW Cell 2 — Load & normalize UHC + GA WC (no TIN/provider filters), compute quick WC benchmark

import pandas as pd
import numpy as np

# --- Load ---
uhc = pd.read_csv(UHC_CSV, low_memory=False)
wc  = pd.read_csv(GA_WC_FLAT, low_memory=False)

# --- UHC: normalize to professional CPT, aggregate by code ---
uhc_norm = uhc.copy()

# code + rate
uhc_norm["code"] = (
    uhc_norm.get("billing_code", uhc_norm.get("code", pd.Series(index=uhc_norm.index, dtype="object")))
    .astype(str)
    .str.strip()
)
rate_col = "negotiated_rate" if "negotiated_rate" in uhc_norm.columns else ("price" if "price" in uhc_norm.columns else None)
if rate_col is None:
    raise ValueError("Could not find a negotiated rate column in UHC data.")
uhc_norm["negotiated_rate"] = pd.to_numeric(uhc_norm[rate_col], errors="coerce")

# best-effort filters (if columns exist)
uhc_mask = pd.Series(True, index=uhc_norm.index)
if "billing_class" in uhc_norm.columns:
    uhc_mask &= uhc_norm["billing_class"].astype(str).str.lower().eq("professional")
if "billing_code_type" in uhc_norm.columns:
    uhc_mask &= uhc_norm["billing_code_type"].astype(str).str.upper().eq("CPT")

uhc_prof = uhc_norm[uhc_mask].copy()
uhc_prof = uhc_prof.replace({"code": {"nan": np.nan}}).dropna(subset=["code", "negotiated_rate"])

# aggregate across ALL providers (no TIN/provider filters)
uhc_agg = (
    uhc_prof.groupby("code", as_index=False)
            .agg(uhc_rate_median=("negotiated_rate","median"),
                 uhc_rate_min   =("negotiated_rate","min"),
                 uhc_rate_max   =("negotiated_rate","max"),
                 uhc_cnt        =("negotiated_rate","size"))
)

# --- GA WC: normalize & derive a "global-equivalent" rate per code ---
wc_norm = wc.copy()

# basic filters
wc_mask = pd.Series(True, index=wc_norm.index)
if "state_code" in wc_norm.columns:
    wc_mask &= wc_norm["state_code"].astype(str).str.upper().eq("GA")
if "code_set" in wc_norm.columns:
    wc_mask &= wc_norm["code_set"].astype(str).str.upper().eq("CPT")
if "billing_class" in wc_norm.columns:
    wc_mask &= wc_norm["billing_class"].astype(str).str.lower().eq("professional")

wc_sub = wc_norm[wc_mask].copy()

# normalize fields
wc_sub["code"] = wc_sub["code"].astype(str).str.strip()
wc_sub["price"] = pd.to_numeric(wc_sub.get("price", pd.Series(index=wc_sub.index, dtype="float")), errors="coerce")

# component normalization
def _norm_comp(x: str) -> str:
    x = str(x).strip().lower()
    if x in ("global","g","-",""): return "global"
    if x in ("professional","pro","pc","26"): return "professional"
    if x in ("technical","tech","tc"): return "technical"
    return x

if "component" in wc_sub.columns:
    wc_sub["component_norm"] = wc_sub["component"].map(_norm_comp)
else:
    wc_sub["component_norm"] = "global"

# choose GA WC rate per code with sensible fallback:
# 1) prefer explicit global
# 2) else sum professional + technical if both exist
# 3) else take whichever single component exists
def pick_wc_rate(group: pd.DataFrame) -> pd.Series:
    g = group[group["component_norm"].eq("global")]["price"].dropna()
    if len(g):
        return pd.Series({"ga_wc_rate": g.iloc[0], "ga_wc_basis": "global"})
    pc = group[group["component_norm"].eq("professional")]["price"].dropna()
    tc = group[group["component_norm"].eq("technical")]["price"].dropna()
    if len(pc) and len(tc):
        return pd.Series({"ga_wc_rate": pc.iloc[0] + tc.iloc[0], "ga_wc_basis": "pc+tc"})
    if len(pc):
        return pd.Series({"ga_wc_rate": pc.iloc[0], "ga_wc_basis": "professional_only"})
    if len(tc):
        return pd.Series({"ga_wc_rate": tc.iloc[0], "ga_wc_basis": "technical_only"})
    return pd.Series({"ga_wc_rate": np.nan, "ga_wc_basis": "unavailable"})

wc_rates = (
    wc_sub.dropna(subset=["code"])
          .groupby("code", as_index=False)
          .apply(pick_wc_rate)
          .reset_index(drop=True)
)

# --- Merge & metrics ---
bench = uhc_agg.merge(wc_rates, on="code", how="left")
bench["pct_of_ga_wc"] = bench["uhc_rate_median"] / bench["ga_wc_rate"]

# final tidy view
bench = bench[[
    "code",
    "uhc_cnt","uhc_rate_min","uhc_rate_median","uhc_rate_max",
    "ga_wc_rate","ga_wc_basis",
    "pct_of_ga_wc"
]].sort_values(["code"])

bench.head(20)


C:\Users\ChristopherCato\AppData\Local\Temp\ipykernel_35020\2273436185.py:95: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(pick_wc_rate)


,code,uhc_cnt,uhc_rate_min,uhc_rate_median,uhc_rate_max,ga_wc_rate,ga_wc_basis,pct_of_ga_wc
0,0001A,29,70.12,70.12,70.12,NaN,NaN,NaN
1,0001U,29,1093.02,1093.02,1093.02,NaN,NaN,NaN
2,0002A,29,70.12,70.12,70.12,NaN,NaN,NaN
3,0002M,29,764.20,764.20,764.20,NaN,NaN,NaN
4,0002U,29,37.95,37.95,37.95,NaN,NaN,NaN
5,0003A,29,70.12,70.12,70.12,NaN,NaN,NaN
6,0003M,29,764.20,764.20,764.20,NaN,NaN,NaN
7,0003U,29,1442.18,1442.18,1442.18,NaN,NaN,NaN
8,0004A,29,70.12,70.12,70.12,NaN,NaN,NaN
9,0004M,29,119.93,119.93,119.93,NaN,NaN,NaN


In [4]:
# Cell — Simple overlap + non-overlap lists (normalized)

import pandas as pd
import numpy as np

def norm_code(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.strip().str.upper()
    s = s.str.replace(r"[^A-Z0-9]", "", regex=True)   # drop spaces/dashes/etc
    is_digits = s.str.fullmatch(r"\d+")               # pure digits → pad to 5 for CPT
    s = s.where(~is_digits, s.str.zfill(5))
    return s

uhc_codes = norm_code(uhc["billing_code"]).replace({"", "NAN"}, pd.NA).dropna().unique()
wc_codes  = norm_code(wc["code"]).replace({"", "NAN"}, pd.NA).dropna().unique()

set_uhc = set(uhc_codes)
set_wc  = set(wc_codes)
inter   = set_uhc & set_wc
uhc_only = sorted(list(set_uhc - set_wc))
wc_only  = sorted(list(set_wc - set_uhc))

a, b, i = len(set_uhc), len(set_wc), len(inter)
pct_a = (i / a * 100) if a else 0.0
pct_b = (i / b * 100) if b else 0.0

print(f"UHC unique billing_code: {a:,}")
print(f"GA WC unique code:       {b:,}")
print(f"Overlap: {i:,}  ({pct_a:.1f}% of UHC, {pct_b:.1f}% of GA WC)")

print(f"\nUHC-only codes: {len(uhc_only):,}  (showing first 25)")
print(uhc_only[:25])

print(f"\nGA WC-only codes: {len(wc_only):,}  (showing first 25)")
print(wc_only[:25])


UHC unique billing_code: 6,420
GA WC unique code:       11,181
Overlap: 5,583  (87.0% of UHC, 49.9% of GA WC)

UHC-only codes: 837  (showing first 25)
['0001A', '0001U', '0002A', '0002M', '0002U', '0003A', '0003M', '0003U', '0004A', '0004M', '0005U', '0006M', '0007M', '0007U', '0008U', '0009U', '0010U', '0011A', '0011M', '0011U', '0012A', '0012M', '0012U', '0013A', '0013M']

GA WC-only codes: 5,598  (showing first 25)
['00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008', '00010', '00011', '00012', '00013', '00014', '00016', '00017', '00018', '00019', '00020', '00021', '00022', '00023', '00024', '00025', '00026', '00027']


In [5]:
# Simple merge of matching codes (UHC.billing_code ↔ WC.code)

import pandas as pd

# light normalization
norm = lambda s: (s.astype(str).str.strip().str.upper()
                    .str.replace(r"[^A-Z0-9]", "", regex=True)
                    .map(lambda x: x.zfill(5) if x.isdigit() else x))

# prepare minimal fields
uhc_min = uhc[['billing_code', 'negotiated_rate']].copy()
wc_min  = wc[['code', 'price']].copy()

# normalize + numeric
uhc_min['code_norm'] = norm(uhc_min['billing_code'])
uhc_min['negotiated_rate'] = pd.to_numeric(uhc_min['negotiated_rate'], errors='coerce')
wc_min['code_norm'] = norm(wc_min['code'])
wc_min['price'] = pd.to_numeric(wc_min['price'], errors='coerce')

# aggregate UHC across rows per code (median is a good summary)
uhc_agg = uhc_min.dropna(subset=['code_norm','negotiated_rate']) \
                 .groupby('code_norm', as_index=False)['negotiated_rate'].median()

# take first WC price per code (or switch to mean/median if you prefer)
wc_agg = wc_min.dropna(subset=['code_norm','price']) \
               .drop_duplicates(subset=['code_norm'])

# inner-join = matching codes only
merged_df = uhc_agg.merge(wc_agg[['code_norm','price']], on='code_norm', how='inner') \
                   .rename(columns={'code_norm':'code',
                                    'negotiated_rate':'uhc_rate_median',
                                    'price':'ga_wc_rate'})

# quick ratio
merged_df['pct_of_ga_wc'] = merged_df['uhc_rate_median'] / merged_df['ga_wc_rate']

display(merged_df.head(2))


,code,uhc_rate_median,ga_wc_rate,pct_of_ga_wc
0,10004,97.33,104.23,0.933800
1,10005,202.77,267.55,0.757877


In [6]:
uhc_min['code_norm'] = norm(uhc_min['billing_code'])
uhc_min['negotiated_rate'] = pd.to_numeric(uhc_min['negotiated_rate'], errors='coerce')
wc_min['code_norm'] = norm(wc_min['code'])
wc_min['price'] = pd.to_numeric(wc_min['price'], errors='coerce')

In [7]:
# Cell — Build full UHC + GA WC dataframes with GA WC rate mapped in and % of GA rate

import pandas as pd

def norm_code(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.strip().str.upper()
    s = s.str.replace(r"[^A-Z0-9]", "", regex=True)
    is_digits = s.str.fullmatch(r"\d+")
    return s.where(~is_digits, s.str.zfill(5))

# --- Prep GA WC: per-code reference rate (median) ---
wc_full = wc.copy()
wc_full["code_norm"] = norm_code(wc_full["code"])
wc_full["price"] = pd.to_numeric(wc_full["price"], errors="coerce")
wc_rate_map = (
    wc_full.dropna(subset=["code_norm","price"])
           .groupby("code_norm")["price"]
           .median()
           .to_dict()
)

# --- Enhance UHC with GA WC rate + % of GA rate ---
uhc_full = uhc.copy()
uhc_full["code_norm"] = norm_code(uhc_full["billing_code"])
uhc_full["negotiated_rate"] = pd.to_numeric(uhc_full["negotiated_rate"], errors="coerce")
uhc_full["ga_wc_rate"] = uhc_full["code_norm"].map(wc_rate_map)
uhc_full["pct_of_ga_wc"] = (uhc_full["negotiated_rate"] / uhc_full["ga_wc_rate"]) * 100
uhc_full["source"] = "UHC"

# --- Enhance GA WC rows similarly (their own % will be around 100 if same as median) ---
wc_full["ga_wc_rate"] = wc_full["code_norm"].map(wc_rate_map)
wc_full["pct_of_ga_wc"] = (wc_full["price"] / wc_full["ga_wc_rate"]) * 100
wc_full["source"] = "GA_WC"

# --- Combine full rows from both sources (matching & non-matching codes included; pct will be NaN where no match) ---
combined_df = pd.concat([uhc_full, wc_full], ignore_index=True, sort=False)

# Optional: keep only rows where a GA WC reference exists
# combined_df = combined_df[combined_df["ga_wc_rate"].notna()].copy()

combined_df.head(2)


,Unnamed: 0,provider_group_id,npi,tin_type,tin_value,reporting_entity_name_x,reporting_entity_type_x,last_updated_on_x,version_x,provider_reference_id,...,modifier,component,site_of_service,apc_code,ms_drg_code,si,pi,fud,price,price_note
0,0.0,168.0,1.700900e+09,ein,201354399.0,UnitedHealthcare of Georgia Inc.,Insurer,2025-08-01,1.0.0,168.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,168.0,1.700900e+09,ein,201354399.0,UnitedHealthcare of Georgia Inc.,Insurer,2025-08-01,1.0.0,168.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# All sources (UHC + GA_WC) but only for codes that mapped to a GA WC rate
matches = combined_df[combined_df["ga_wc_rate"].notna()].copy()

# If you want ONLY the UHC rows that match GA WC:
uhc_matches = matches[matches["source"] == "UHC"].copy()

matches.head(2)
matches.columns
matches.to_csv("matches.csv", index=False)


In [12]:
# simplest: keep it as a Series, de-dupe, sort, then save
uhc[['description']].drop_duplicates().dropna().sort_values('description') \
    .to_csv('unique_desc.csv', index=False)
